In [ ]:
import math

def get_tasks_for_thread(remainder: int, num_tasks_default: int, thread: int):
    num_tasks = num_tasks_default + 1 if thread < remainder else num_tasks_default
    threads_default = max(thread - remainder, 0)
    threads_default_plus = thread - threads_default
    start = threads_default * num_tasks_default + threads_default_plus * (num_tasks_default + 1)
    end = start + num_tasks - 1
    return num_tasks, start, end

def divide_tasks(tasks: int, threads: int):
    remainder = tasks % threads
    num_tasks_default = math.floor(tasks / threads)
    for thread in range(threads):
        num_tasks, start, end = get_tasks_for_thread(remainder, num_tasks_default, thread)
        print(f"Thread {thread}: tasks {start}-{end}, i.e. {num_tasks} tasks.")        

In [15]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-dark')

api = wandb.Api()
entity = "joasiee"
instances = [16, 23, 17, 13, 6]
optims = ["GOMEA", "AdaptiveStochasticGradientDescent"]

def get_runs(project, filters={}):
    return api.runs(entity + "/" + project, filters=filters)

def get_finished_run_data(runs):
    names, configs, histories = [], [], []
    for run in runs:
        if run.state == "finished":
            names.append(run.name)
            configs.append(
                {k: v for k, v in run.config.items()
                 if not k.startswith('_')})
            history = run.history()
            histories.append(history)
    return names, configs, histories

def find_duplicates(project, weight0_lim=0.11, weight1_lim=0.21):
    duplicate_cases = 0
    for instance in instances:
        for optim in optims:
            for weight0 in np.arange(0.01, weight0_lim, 0.01):
                for weight1 in np.arange(0.01, weight1_lim, 0.01):
                    weight0 = np.around(weight0, 2)
                    weight1 = np.around(weight1, 2)
                    filters = {
                        "state": {"$in": ["finished", "running"]},
                        "config.Metric0Weight": float(weight0),
                        "config.Metric1Weight": float(weight1),
                        "config.Optimizer": optim,
                        "config.Instance": instance
                    }
                    runs = api.runs(entity + "/" + project, filters={"state": {"$in": ["finished", "running"]}, "config.Metric0Weight": float(weight0), "config.Metric1Weight": float(weight1)})
                    if len(runs) > 1:
                        duplicate_cases += 1
                        print(f"{len(runs)} duplicates found for {filters}")
    print(f"Found {duplicate_cases} in total.")

def pareto_plot(histories):


In [16]:
project = "pareto_front"
runs = get_runs(project)
names, configs, histories = get_finished_run_data(runs)

,R0/||gradient1||,_step,R0/metric0,R0/3a:time,R0/time1[ms],R0/metric1,R0/||gradient||,_runtime,R0/time0[ms],R0/metric,...,R1/metric1,R1/3a:time,R1/missedpixels,R1/time0[ms],R1/metric0,R1/||gradient0||,R1/||gradient||,R1/time1[ms],R1/metric,R1/||gradient1||
0,0.000002,12,0.055253,3.739027,0.2,0.000068,0.000104,6,0.1,0.005539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.000003,19,0.053571,6.613552,0.2,0.000091,0.000107,6,0.1,0.005375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.000003,33,0.078048,14.833826,0.2,0.000126,0.000126,8,0.1,0.007830,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000003,40,0.063088,19.248276,0.2,0.000127,0.000094,9,0.1,0.006334,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.000004,58,0.048496,30.528748,0.1,0.000155,0.000093,10,0.1,0.004881,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
